# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [39]:
import numpy as np

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
from inauguralproject import HouseholdSpecializationModelClass


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Question 1

**Explain how you solve the model**

In [40]:
# code for solving the model (remember documentation and comments)
# We define the function of (sol.HF_vec/sol.HM_vec)

NameError: name 'y' is not defined

# Question 2

Explain your code and procedure

In [ ]:
# code

# Question 3

Explain your code and procedure

In [ ]:
# code

ADD CONCISE CONLUSION.